# Terminology

- History / Schedule - order of execution for a set of transactions
- Tn - resembles a specific transaction place in a history
- R(x) - `read` operation on object `x`
- W(x) - `write` operation on object `x`
- Com. - commit transaction
- Abort - abort transaction

# Problem

## Idea

When concurrent transaction are executed in the same time, naturally *isolation* and *consistency* is hard to achieve.

## Serial Example

T1	        |   T2  
--          |   --  
R(X)        |       
W(X)        |       
Com.        |       
&#xfeff;    |   R(Y)
&#xfeff;    |   W(Y)
&#xfeff;    |   Com.

## Concurrent Example

Think what can go wrong here in terms of consistency and isolation

T1	        |   T2  
--          |   --  
R(X)        |       
&#xfeff;    |   R(X)
&#xfeff;    |   W(X)
W(X)        |       
&#xfeff;    |   Com.
Com.        |       

# Solution

## ANSI SQL Standard

ANSI described some isolation levels that have to be implemented to deal with concurrency.

They are based on the different *Read Phenomenons* that they allow in terms of concurrent transactions.

### Read Phenomenons

#### Dirty Read

**See uncommitted changes**

Occurs when a transaction retrieves a row that has been updated by another transaction that is not yet committed.

<img src="./helpers/dirty-read.png" alt="drawing" width="900"/>

#### Non-Repeatable Read

**See committed changes on row between two queries**

Occurs when a transaction retrieves a row twice and that row is updated by another transaction that is committed in between

<img src="./helpers/non-repeatable-read.png" alt="drawing" width="900"/>

#### Phantom Read

**See committed changes on result set between two queries**

Occurs when a transaction retrieves a set of rows twice and new rows are inserted into or removed from that set by another transaction that is committed in between

<img src="./helpers/phantom-read.png" alt="drawing" width="900"/>

### Isolation Levels

Isolation Level	| Dirty Read  |	Nonrepeatable Read  |	Phantom Read    |	Serialization Anomaly
---------       | -------     | -------             |   ---------       |   -------
Read uncommitted    |	Allowed, but not in PG  |	Possible    |	Possible    |	Possible
Read committed  |	Not possible    |	Possible    |	Possible    |	Possible
Repeatable read |	Not possible    |	Not possible    |	Allowed, but not in PG  |	Possible
Serializable    |	Not possible    |	Not possible    |	Not possible    |   Not possible

In PostgreSQL:
- **Read Uncommitted** mode behaves like **Read Committed** and it's the *default*!
- **Repeatable Read** mode behaves like **Serializable** except Snapshot Isolation anomalies.

## Integrity Rules

Integrity rules are abstract properties of a transaction system that provide the building blocks of the isolation level solution

### Serializable

#### Definition

A schedule is serializable if it is equivalent in its outcome to a serial schedule.

#### Idea

The order of dependent **operations** should persist.

#### Example

T1	        |   T2  
--          |   --  
R(X)        |       
W(X)        |       
&#xfeff;    |   R(Y)
&#xfeff;    |   W(Y)
&#xfeff;    |   Com.
Com.        |       

### Recoverable

#### Definition

A schedule is recoverable if transactions only commit after all transactions whose changes they read have committed.


#### Idea

A rollback is affecting all dependent transactions.

#### Example

T1	        |   T2  
--          |   --  
&#xfeff;    |   W(X)       
W(Y)        |   
W(X)        |   
&#xfeff;    |   R(Y)
Com.        |       
&#xfeff;    |   Com.

## Concurrency Control Solution Categories

- Optimistic - No locks, only check for violations of the integrity rules at each transaction's commit. If violations are detected upon a transaction's commit, the transaction is aborted and restarted. This approach is very efficient when few transactions are aborted.
- Pessimistic - Use locks, block an operation of a transaction, if it may cause violation of the rules, until the possibility of violation disappears. Blocking operations is typically involved with performance reduction.
- Semi-optimistic - Responds pessimistically or optimistically depending on the type of violation and how quickly it can be detected.

## MVCC

### Definition

Multi Version Concurrency Control (MVCC) - is mostly an optimistic concurrency control method that reduces the possible conflicts by creating multiple versions of database objects.

*Although locks are used in MVCC, read locks are not blocking write locks!*

### Snapshot Isolation - MVCC Implementation

Queries are viewing a specific point in time snapshot of the database in a consistent state that doesn't change during the query execution.